In [1]:
import os
import json

import numpy as np
import torch

In [2]:
def ensemble(score_dict, weight_dict, labels, softmax=False):
    resemble_score = None
    for k in weight_dict:
        tmp = score_dict[k]
        if softmax:
            tmp = torch.from_numpy(tmp)
            tmp = torch.softmax(tmp, dim=1).numpy()
        resemble_score = weight_dict[k] * tmp if resemble_score is None \
        else weight_dict[k] * tmp + resemble_score
    acc = np.sum(np.argmax(resemble_score, axis=1) == labels) / len(labels)
    return resemble_score, acc

In [8]:
# b = 1
# r = 0.8
# softmax = False
# print("  a  |       |  AM   | diff")
# for a in np.linspace(0, 1, 21):
#     weight_dict = {
#         "j": b,
#         "b": b,
#         "jm": a,
#         "bm": a,
#     }
#     _, acc = ensemble(score_dict, weight_dict, labels, softmax)

#     weight_dict_am = {
#         "j": b,
#         "b": b,
#         "jm": a,
#         "bm": a,
#         "jmam": a * r,
#         "bmam": a * r,
#     }
#     _, acc_am = ensemble(score_dict, weight_dict_am, labels, softmax)
#     diff = acc_am - acc
#     print(f"{a:0.2f} | {acc*100:0.2f} | {acc_am*100:0.2f} | {diff*100:0.2f}")

In [41]:
def get_results(dataset, ckp_type, combinations):
    ckp_type = "best_acc"
    score_dir = f"score/{dataset}/{ckp_type}"
    log_dir = f"log/{dataset}"

    score_dict = {}
    for file in os.scandir(score_dir):
        if file.path.split(".")[-1] != "npy":
            continue
        name = file.name.split(".")[0]
        name = "_".join(name.split("_")[2:-2])[:-4]
        score = np.load(file.path)
        score_dict[name] = score
        
    log_dict = {}
    for file in os.scandir(log_dir):
        if file.path.split(".")[-1] != "json":
            continue
        name = file.name.split(".")[0]
        name = "_".join(name.split("_")[2:-1])[:-4]
        with open(file.path, "r") as f:
            log_data = json.load(f)
        log_dict[name] = log_data
        
    labels = list(log_dict.values())[0]["labels"]
    
    results = {}
    for c in combinations:
        name = c["name"]
        weight_dict = c["weight_dict"]
        _, acc = ensemble(score_dict, weight_dict, labels)
        results[name] = f"{acc*100:.2f}"
    return results
        

In [42]:
combinations = [
    {
        "name": "J",
        "weight_dict": {"j": 1},
    },
    {
        "name": "B",
        "weight_dict": {"b": 1},
    },
    {
        "name": "JM",
        "weight_dict": {"jm": 1},
    },
    {
        "name": "BM",
        "weight_dict": {"bm": 1},
    },
    {
        "name": "JM+AM",
        "weight_dict": {"jmam": 1},
    },
    {
        "name": "BM+AM",
        "weight_dict": {"bmam": 1},
    },
    {
        "name": "E(J,B)",
        "weight_dict": {"j": 1, "b": 1},
    },
    {
        "name": "E(J,B,JM,BM)",
        "weight_dict": {"j": 1, "b": 1, "jm": 0.5, "bm": 0.5},
    },
    {
        "name": "E(J,B,JM+AM,BM+AM)",
        "weight_dict": {"j": 1, "b": 1, "jmam": 0.5, "bmam": 0.5},
    },
    {
        "name": "E(J,B,JM,BM,JM+AM,BM+AM)",
        "weight_dict": {"j": 1, "b": 1, "jm": 0.5, "bm": 0.5, "jmam": 0.4, "bmam": 0.4},
    },
]

In [43]:
def print_format(headers, format_dict, latex=False):
    if latex:
        table_format = """
\\hline
{header} 
\\hline\\hline
{body}
\\hline
"""
        header_str = " & ".join(headers) + " \\\\"
        row_arr = []
        bold = [list(format_dict.keys())[0] for _ in range(len(headers) - 1)]
        for k in format_dict:
            for j in range(len(format_dict[k])):
                if float(format_dict[k][j]) > float(format_dict[bold[j]][j]):
                    bold[j] = k
        ob = "{"
        cb = "}"
        for j, k in enumerate(bold):
            format_dict[k][j] = f"\\textbf{ob}{format_dict[k][j]}{cb}"
        for k in format_dict:
            row_arr.append(f"{k} & {' & '.join(format_dict[k])} \\\\")
        body_str = "\n".join(row_arr)
        return table_format.format(header=header_str, body=body_str)
        

In [44]:
ckp_type = "best_acc"
format_dict = {}
for c in combinations:
    format_dict[c["name"]] = []
headers = ["Method"]
for dataset in ["ucla", "ntu-x-view", "ntu-x-subject"]:
    headers.append(dataset)
    results = get_results(dataset, ckp_type, combinations)
    for k in results:
        format_dict[k].append(results[k])

j
bmam
jm
bm
jmam
b
j
bmam
jm
bm
jmam
b
j
bmam
jm
bm
jmam
b


In [45]:
print(print_format(headers, format_dict, True))


\hline
Method & ucla & ntu-x-view & ntu-x-subject \\ 
\hline\hline
J & 92.03 & 92.93 & 87.57 \\
B & 91.81 & 94.89 & 90.13 \\
JM & 93.10 & 93.73 & 87.08 \\
BM & 89.66 & 92.44 & 86.75 \\
JM+AM & 91.38 & 94.15 & 87.72 \\
BM+AM & 90.52 & 93.05 & 87.65 \\
E(J,B) & 95.26 & 96.11 & 91.39 \\
E(J,B,JM,BM) & 95.91 & 96.61 & 91.95 \\
E(J,B,JM+AM,BM+AM) & 95.26 & 96.60 & 92.10 \\
E(J,B,JM,BM,JM+AM,BM+AM) & \textbf{96.12} & \textbf{96.77} & \textbf{92.16} \\
\hline

